In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_attention_functions import seqsignet_attention_hyperparameter_search

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-09-26 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-26 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-26 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-26 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-26 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

# history_length=11

In [10]:
shift = 3
window_size = 5
n = 3

In [12]:
(
    seqsignet_network_umap_kfold_11,
    best_seqsignet_network_umap_kfold_11,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'best_model_2023-09-26-06-49-26-601090.pkl'

In [ ]:
seqsignet_network_umap_kfold_11

In [ ]:
best_seqsignet_network_umap_kfold_11["f1"].mean()

In [ ]:
best_seqsignet_network_umap_kfold_11["precision"].mean()

In [ ]:
best_seqsignet_network_umap_kfold_11["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_11["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    seqsignet_network_grp_kfold_11,
    best_seqsignet_network_grp_kfold_11,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_grp_kfold_11

In [ ]:
best_seqsignet_network_grp_kfold_11["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_11["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_11["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_11["recall_scores"]).mean(axis=0)

# history_length=20

In [ ]:
shift = 3
window_size = 5
n = 6

In [ ]:
(
    seqsignet_network_umap_kfold_20,
    best_seqsignet_network_umap_kfold_20,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_umap_kfold_20

In [ ]:
best_seqsignet_network_umap_kfold_20["f1"].mean()

In [ ]:
best_seqsignet_network_umap_kfold_20["precision"].mean()

In [ ]:
best_seqsignet_network_umap_kfold_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_20["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    seqsignet_network_grp_kfold_20,
    best_seqsignet_network_grp_kfold_20,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_grp_kfold_20

In [ ]:
best_seqsignet_network_grp_kfold_20["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_20["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_20["recall_scores"]).mean(axis=0)

# history_length=35

In [ ]:
shift = 3
window_size = 5
n = 11

In [ ]:
(
    seqsignet_network_umap_kfold_35,
    best_seqsignet_network_umap_kfold_35,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    split_ids=client_transcript_id,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_umap_kfold_35

In [ ]:
best_seqsignet_network_umap_kfold_35["f1"].mean()

In [ ]:
best_seqsignet_network_umap_kfold_35["precision"].mean()

In [ ]:
best_seqsignet_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_kfold_35["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    seqsignet_network_grp_kfold_35,
    best_seqsignet_network_grp_kfold_35,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    split_ids=client_transcript_id,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_grp_kfold_35

In [ ]:
best_seqsignet_network_grp_kfold_35["f1"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_35["precision"].mean()

In [ ]:
best_seqsignet_network_grp_kfold_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_kfold_35["recall_scores"]).mean(axis=0)